In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
train_data_dir='/content/gdrive/My Drive/lung_colon_image_set/colon_image_sets'



In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=45,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=45,
    class_mode='binary',
    subset='validation') # set as validation data


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# example of tending the resNet50 model
from tensorflow.keras.applications import ResNet50
import keras
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model = ResNet50(include_top=False, input_shape=(150, 150, 3))
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(2, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer= opt, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

94773248/94765736 [==============================] - 5s 0us/step


In [ ]:
# simple early stopping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=50)
mc = ModelCheckpoint('/content/gdrive/My Drive/model/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


In [ ]:
batch_size=32
nb_epochs=1000
history=model.fit(
    train_generator,
    batch_size=32,
    validation_data = validation_generator,

    epochs = nb_epochs,callbacks=[es],workers=10)

Epoch 1/1000
178/178 [==============================] - 2886s 16s/step - loss: 0.8962 - accuracy: 0.9392 - val_loss: 4.6959 - val_accuracy: 0.5000
Epoch 2/1000
178/178 [==============================] - 2829s 16s/step - loss: 1.0520 - accuracy: 0.9193 - val_loss: 6530.0151 - val_accuracy: 0.5000
Epoch 3/1000
178/178 [==============================] - 2827s 16s/step - loss: 0.1336 - accuracy: 0.9485 - val_loss: 5.8479 - val_accuracy: 0.5000
Epoch 4/1000
178/178 [==============================] - 2834s 16s/step - loss: 0.0728 - accuracy: 0.9744 - val_loss: 4.9690 - val_accuracy: 0.5000
Epoch 5/1000
178/178 [==============================] - 2823s 16s/step - loss: 0.0505 - accuracy: 0.9834 - val_loss: 8.5639 - val_accuracy: 0.5020
Epoch 6/1000
178/178 [==============================] - 2825s 16s/step - loss: 0.0300 - accuracy: 0.9911 - val_loss: 3.6922 - val_accuracy: 0.6355
Epoch 7/1000
178/178 [==============================] - 2829s 16s/step - loss: 0.0271 - accuracy: 0.9902 - val_loss

In [ ]:
test_directory='/content/gdrive/My Drive/CAT_DOGS/2Class/testing_data'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_directory, # same directory as training data
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')



NameError: ignored

In [ ]:
score = model.evaluate(test_generator)

13/13 [==============================] - 178s 14s/step - loss: 1.1437 - accuracy: 0.5000
